In [18]:
import numpy
import re
import math
import glob
#from nb_model import NB_model 



In [19]:
vocab = []
hamwords = {}
hamdoc_count = 0
hamfiles = [file for file in glob.glob("./train/*ham*.txt")]
stopwords = ['return', 'path', 'spamassassin', 'org', 'to', 'yyyy', 'localhost', 'netnoteinc', 'com', 'received', 'from', 'by', 'labs', 'with', 'esmtp', 'id', 'c', 'for', 'jm', 'wed', 'dogma', 'slashnull', 'g', 'jmason', 'f', 'a', 'd', 'mail', 'smtp', 'subject', 'in', 'reply', 'mime', 'version', 'content', 'type', 'text', 'charset', 'us', 'message', 'x', 'list', 'mailto', 'date', 'tue', 'i', 'm', 'that', 'are', 'the', 'can', 't', 'if', 'is', 'my', 'not', 'an', 'of', 'this', 'and', 's', 'as', 'have', 'be', 'it', 'on', 'or', 'do', 'you', 'at', 'e', 'b', 'http', 'www', 'net', 'jul', 'mon']

for file in hamfiles:
    hamdoc_count  = hamdoc_count+1
    doc = open(file, 'r').read().lower()
    doc = re.sub('[a-z]*@*.com', '', doc)
    doc = re.sub('<*[a-z]*>', '', doc)
    words = [word for word in re.split('[^a-zA-Z]',doc)]
    for word in words:
        if word != '' and word not in stopwords:
            try:
                hamwords[word] += 1
            except KeyError:
                hamwords[word] = 1

spamdoc_count = 0
spamwords = {}
spamfiles = [file for file in glob.glob("./train/*spam*.txt")]
for file in spamfiles:
    spamdoc_count = spamdoc_count+1
    doc = open(file, 'r').read().lower()
    words = [word for word in re.split('[^a-zA-Z]',doc)]
    for word in words:
        if word != '' and word not in stopwords:
            try:
                spamwords[word] += 1
            except KeyError:
                spamwords[word] = 1
    
print(len(vocab))
print(len(spamwords))
print(len(hamwords))

#(vocab)

0
47483
19693


In [20]:
stopwords = ['to', 'from', 'the', 'by', 'with', 'for', 'a', 'i', 'of', 'and', 'in', 'that', 'is', 'it', 'on', 'you', 'be', 'this',
            'have', 'as', 'are', 'at', 'or', 'us', 'my', 'so', 'an', 'will', 'your', 'no', 'any', 'cc', 'than', 'he', 'she', 'am',
            'com', 'org', 'net', 'received', 'http', 'subject', 'mailto', 'mail', 'www', 'delivered', 'but', 'localhost', 
            'we', 'd', 'b', 'p', 'e', 'id', 'x', 'f', 'g', 's', '']

#for elem in sorted(spamwords.items() ,  key=lambda x: x[1], reverse = True) :
#        print(elem[0] , " ::" , elem[1] )
stopwords2 = []

for key in hamwords.keys():
    if key in spamwords.keys():
        if hamwords[key] > 700 and spamwords[key] > 700:
            stopwords2.append(key)
            print(key , hamwords[key])
print(stopwords2)            

[]


In [21]:
class NB_model:
    
    def __init__(self,msg):
        self.spam_prob_dict ={}
        self.ham_prob_dict = {}
        self.prob_spam = 0
        self.prob_ham = 0
        self.tot_ham_words = 0
        self.tot_spam_words = 0
        self.stopwords = ['to', 'from', 'the', 'by', 'with', 'for', 'a', 'i', 'of', 'and', 'in', 'that', 'is', 'it', 'on', 'you', 'be', 'this',
            'have', 'as', 'are', 'at', 'or', 'us', 'my', 'so', 'an', 'will', 'your', 'no', 'any', 'cc', 'than', 'he', 'she', 'am',
            'com', 'org', 'net', 'received', 'http', 'subject', 'mailto', 'mail', 'www', 'delivered', 'but', 'localhot', 
            'we', 'd', 'b', 'p', 'e']
        self.stopwords2 = ['return', 'path', 'spamassassin', 'org', 'to', 'yyyy', 'localhost', 'snetnoteinc', 'com', 'received', 'from', 'by', 'labs', 'with', 'esmtp', 'id', 'c', 'for', 'jm', 'wed', 'dogma', 'slashnull', 'g', 'jmason', 'f', 'a', 'd', 'mail', 'smtp', 'subject', 'in', 'reply', 'mime', 'version', 'content', 'type', 'text', 'charset', 'us', 'message', 'x', 'list', 'mailto', 'date', 'tue', 'i', 'm', 'that', 'are', 'the', 'can', 't', 'if', 'is', 'my', 'not', 'an', 'of', 'this', 'and', 's', 'as', 'have', 'be', 'it', 'on', 'or', 'do', 'you', 'at', 'e', 'b', 'http', 'www', 'net', 'jul', 'mon']

        print(msg)
        
    def train(self, hm_dict, sp_dict, hm_prob, sp_prob):
        vocab_set = []
        self.prob_spam = sp_prob
        self.prob_ham = hm_prob
        for key,value in hm_dict.items():
            self.tot_ham_words += value
            vocab_set.append(key)
        vocab_set = set(vocab_set)
    
        print(len(vocab_set))
        for key,value in spamwords.items():
            self.tot_spam_words += value
            vocab_set.add(key)
        print(len(vocab_set))
        self.vocab = list(vocab_set)
        print(len(self.vocab))
        file = open("./model.txt", 'w+')
    
        index = 1
        for word in sorted(self.vocab):
            try:
                hm_dict[word] += 0
            except KeyError:
                hm_dict[word] = 0
                
            try:
                sp_dict[word] += 0
            except KeyError:
                sp_dict[word] = 0
                
            prob_word_ham = (hm_dict[word]+0.5)/(self.tot_ham_words+len(self.vocab))
            self.ham_prob_dict[word] = prob_word_ham
            prob_word_spam = (sp_dict[word]+0.5)/(self.tot_spam_words+len(self.vocab))
            self.spam_prob_dict[word] = prob_word_spam
           # print("%d  %s  %d  %f  %d  %f"%(index, word, hm_dict[word], prob_word_ham, sp_dict[word], prob_word_spam))
            file.write("%d  %s  %d  %f  %d  %f\n"%(index, word, hm_dict[word], prob_word_ham, sp_dict[word], prob_word_spam))
            index +=1
            
        file.close()
        
    def test(self, words):
        score_ham = math.log10(self.prob_ham)
        score_spam = math.log10(self.prob_spam)
        for word in words:
            if word != '' and word not in self.stopwords2:
                try:
                    self.ham_prob_dict[word] += 0
                except KeyError:
                    self.ham_prob_dict[word] = 0.5/(self.tot_ham_words+len(self.vocab))
                    self.spam_prob_dict[word] = 0.5/(self.tot_spam_words+len(self.vocab))
                
                score_ham = score_ham + math.log10(self.ham_prob_dict[word])
                score_spam = score_spam + math.log10(self.spam_prob_dict[word])              
            
        if score_ham > score_spam:
            return("ham",score_ham,score_spam)
        else:
            return("spam",score_ham,score_spam)
        




In [22]:
prob_ham = (hamdoc_count/(hamdoc_count+spamdoc_count))
prob_spam = (spamdoc_count/(hamdoc_count+spamdoc_count))

model = NB_model("creating object")
model.train(hamwords,spamwords,prob_ham,prob_spam)

creating object
19693
59642
59642


In [23]:
index = 1
y_act = []
y_pred = []
res_file = open("./result.txt", 'w+')
ham_test_files = [file for file in glob.glob("./test/*ham*.txt")]

# positive = ham and negative = spam

tp = 0
tn = 0
fp = 0
fn = 0

for file in ham_test_files:
    doc = open(file, 'r').read().lower()
    words = [word for word in re.split('[^a-zA-Z]',doc)]
    (result,h_score,s_score) = model.test(words)
    y_act.append(0)
    if result == "ham":
        output = 'right'
        tp += 1
        y_pred.append(0)
    else:
        output = 'wrong'
        fn += 1
        y_pred.append(1)
    #print("%d  %s  %s  %f  %f  %s  %s"%(index, file, result, h_score, s_score, "ham", output))
    res_file.write("%d  %s  %s  %f  %f  %s  %s \n"%(index, file, result, h_score, s_score, "ham", output))
    index += 1
    
spam_test_files = [file for file in glob.glob("./test/*spam*.txt")]

for file in spam_test_files:
    doc = open(file, 'r', encoding='cp437').read().lower()
    words = [word for word in re.split('[^a-zA-Z]',doc)]
    (result,h_score,s_score) = model.test(words)
    y_act.append(1)
    if result == "ham":
        output = 'wrong'
        fp += 1
        y_pred.append(0)
    else:
        output = 'right'
        tn += 1
        y_pred.append(1)
    #print("%d  %s  %s  %f  %f  %s  %s"%(index, file, result, h_score, s_score, "spam", output))
    res_file.write("%d  %s  %s  %f  %f  %s  %s \n"%(index, file, result, h_score, s_score, "spam", output))
    index += 1
    
res_file.close()

#resultFn(tp, tn, fp, fn)

In [24]:
def resultFn(true_pos, true_neg, false_pos, false_neg):
    print("-------------------------------------------Evaluation--------------------------------------------")
    print("true_pos : "+ str(true_pos) +"\ntrue_neg : "+str(true_neg)+"\nfalse_pos : "+ str(false_pos) +"\nfalse_neg : "+ str(false_neg) +"\n")
    precision = true_pos/(true_pos + false_pos)
    recall = true_pos/(true_pos + false_neg)
    f1 = 2 * ((precision * recall)/(precision + recall))
    accuracy = (true_pos + true_neg)/(true_pos + false_neg + true_neg + false_pos)
    print("Precision : "+str(precision*100) + "\nRecall : "+str(recall*100)+"\nF1-score : "+str(f1*100)+"\nAccuracy : "+str(accuracy*100))

In [25]:
resultFn(tp, tn, fp, fn)

-------------------------------------------Evaluation--------------------------------------------
true_pos : 391
true_neg : 363
false_pos : 37
false_neg : 9

Precision : 91.35514018691589
Recall : 97.75
F1-score : 94.44444444444444
Accuracy : 94.25
